In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
from pathlib import Path

import matplotlib.pyplot as plt
import numpy as np

import src.assumptions as A
from src import (
    demand_model,
    matplotlib_style,  # noqa: F401
    power_system,
    supply_model,
)
from src.units import Units as U

# storage model plots

In [ ]:
renewable_capacity = 400
demand_mode = "cb7"
demand_df = demand_model.predicted_demand(mode=demand_mode, average_year=False)
df = supply_model.get_net_supply(demand_df).reset_index()

storage = power_system.PowerSystem(
    renewable_capacity=renewable_capacity * U.GW,
    hydrogen_storage_capacity=A.HydrogenStorage.CavernStorage.MaxCapacity,
    electrolyser_power=A.HydrogenStorage.Electrolysis.Power,
    dac_capacity=A.DAC.Capacity,
)
sim_df = storage.run_simulation(df)
results = storage.analyze_simulation_results(sim_df)
storage.plot_simulation_results(sim_df, results, demand_mode)

# surface plots


In [ ]:
demand_mode = "cb7"

# A.EnergyDemand2050 = 575 * U.TWh
# A.EnergySystemLosses = 0
A.EnergyDemand2050 = A.CB7EnergyDemand2050
CONTINGENCY_MIN_STORAGE = 20 * U.TWh
renewable_capacities = range(200, 410, 10)
electrolyser_powers = range(20, 110, 10)
max_storage = range(30, 200, 10)
Z = np.zeros((len(renewable_capacities), len(electrolyser_powers))) * np.nan
xyzs = []


demand_df = demand_model.predicted_demand(mode="seasonal", average_year=False)
df = supply_model.get_net_supply(demand_df).reset_index()

total_combinations = len(renewable_capacities) * len(electrolyser_powers) * len(max_storage)

for renewable_capacity in renewable_capacities:
    for electrolyser_power in electrolyser_powers:
        for storage in max_storage:
            model = power_system.PowerSystem(
                renewable_capacity=renewable_capacity * U.GW,
                max_storage_capacity=storage * U.TWh,
                electrolyser_power=electrolyser_power * U.GW,
                dac_capacity=A.DAC.Capacity,
            )

            net_supply_df = model.run_simulation(df.copy())
            results = model.analyze_simulation_results(net_supply_df)
            if results is None:
                continue
            if results["minimum_storage"] < CONTINGENCY_MIN_STORAGE:
                continue

            Z[renewable_capacities.index(renewable_capacity), electrolyser_powers.index(electrolyser_power)] = storage
            break

In [ ]:
# meshgrid for x and y
X, Y = np.meshgrid(electrolyser_powers, renewable_capacities)

# flatten arrays and filter out NaN values for plot_trisurf
X_flat = X.flatten()
Y_flat = Y.flatten()
Z_flat = Z.flatten()

# create mask to remove NaN values
mask = ~np.isnan(Z_flat)
X_clean = X_flat[mask]
Y_clean = Y_flat[mask]
Z_clean = Z_flat[mask]

# plot the surface using trisurf to handle NaN values
fig = plt.figure(figsize=(6, 5), dpi=200, tight_layout=True)
ax = fig.add_subplot(111, projection="3d")
ax.plot_trisurf(X_clean, Y_clean, Z_clean, color="lightblue", alpha=1, edgecolor="black", linewidth=0.1)
ax.set_xlabel("Electrolyser Power (GW)")
ax.set_ylabel("Renewable Capacity (GW)")
ax.set_zlabel("Maximum Storage (TWh)")
ax.view_init(elev=20, azim=40)
# zoom out to make the axis smaller
ax.set_title(
    f"{A.EnergyDemand2050.magnitude:.0f} TWh Demand, {CONTINGENCY_MIN_STORAGE.magnitude} TWh Storage Contingency\nNaive Demand Scaling", fontsize=10
)
plt.tight_layout()
plt.savefig(Path("storage_capacity_surface.png"), dpi=200)
plt.show()

# mt co2

In [ ]:
from src.utils import convert_energy_cost

convert_energy_cost(A.DAC.EnergyCost.Medium, A.MolecularWeightCO2) * 8

In [ ]:
convert_energy_cost(400 * U.kJ / U.mol, A.MolecularWeightCO2)